### Add Directories/ Load Files

In [67]:
import os
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from utils import find_folders

In [68]:
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [69]:
accel_blocks_df = pd.read_excel(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'Accel_Blocks.xlsx'
))

accel_blocks_df.head(10)

,Percept_ID,Blocks,Cond
0,Sub005,1,1
1,Sub005,2,1
2,Sub005,3,2
3,Sub005,4,2
4,Sub005,5,3
5,Sub005,6,3
6,Sub005,7,4
7,Sub006,1,1
8,Sub006,2,1
9,Sub006,3,1


In [70]:
features_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\retap_results\\features'
blocks_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Accelerometer\\retap_results\\extracted_tapblocks'

In [71]:
imp_features = ['freq', 'mean_raise_velocity', 'coefVar_raise_velocity', 'trace_RMSn', 'coefVar_intraTapInt', 'slope_intraTapInt','trace_entropy','jerkiness_trace', 'mean_tapRMS',
                'coefVar_tapRMS', 'mean_impactRMS', 'coefVar_impactRMS', 'slope_impactRMS', 'coefVar_tap_entropy', 'slope_tap_entropy']

In [74]:
all_subs = accel_blocks_df['Percept_ID'].unique()

for sub in all_subs:
    print(sub)

Sub005
Sub006
Sub009
Sub014
Sub029
Sub065
Sub021
Sub025
Sub028
Sub033
Sub050


In [ ]:
for sub in all_subs:

    this_sub = sub

    for cond in np.arange(1,5):

        this_cond = cond

        these_blocks = accel_blocks_df.loc[(accel_blocks_df['Percept_ID'] == this_sub) & (accel_blocks_df['Cond'] == this_cond), 'Blocks']
        block_strings = ['block' + str(num) for num in these_blocks]
        print(block_strings)

        # Filter the json files based on conditions
        jsons_to_import = []
        for filename in os.listdir(features_path):
            if filename.endswith('.json') and this_sub in filename:
                for block_str in block_strings:
                    if filename.endswith(f'{block_str}.json'):
                        jsons_to_import.append(filename)

        print(jsons_to_import)

        # Filter the csv files based on conditions
        csvs_to_import = []
        for filename in os.listdir(blocks_path):
            if filename.endswith('.csv') and this_sub in filename:
                for block_str in block_strings:
                    if filename.endswith(f'{block_str}_250Hz.csv'):
                        csvs_to_import.append(filename)

        print(csvs_to_import)

        # Read the selected JSON files

        combined_dict = []
        for filename in jsons_to_import:
            file_path = os.path.join(features_path, filename)
            with open(file_path) as f:
                this_block_feat = json.load(f)
                matching_keys = set(imp_features) & set(this_block_feat.keys())
                combined_dict.append({key: this_block_feat[key] for key in matching_keys})

        #SAVE IT
        suptitle = str(this_sub) + ' - Condition ' + str(this_cond)

        file_name = "".join(suptitle.split()) + '_features.json'

        # Combine the file name with the current directory to create the file path
        file_path = os.path.join(
            project_path,
            'results',
            'accelerometer',
            'inspections')

        # Serializing json
        json_object = json.dumps(combined_dict, indent=4)
        
        # Writing to sample.json
        with open(os.path.join(file_path,file_name), "w") as outfile:
            outfile.write(json_object)

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(len(csvs_to_import),1, figsize = (18,10))

suptitle = str(this_sub) + ' - Condition ' + str(this_cond)
fig.suptitle(suptitle, fontsize=14, fontweight='bold')

rounded_list = [
    {key: round(value, 2) for key, value in dictionary.items()}
    for dictionary in combined_dict
]

for num, file in enumerate(csvs_to_import):
    this_block_csv = pd.read_csv(os.path.join(blocks_path,file))
    axs[num].plot(this_block_csv)
    axs[num].set_ylabel('Acceleration [g] - ' + str(block_strings[num]))
    axs[num].set_title(rounded_list[num])
    

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'inspections',
    "".join(suptitle.split()),
), dpi = 250
)

### Average all blocks for each condition within subject

In [86]:
json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections', 'Condition4'
)
all_features_cond1 = pd.DataFrame()

# Loop through the JSON files in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        percept_id = filename[:6]

        # Load the JSON file
        with open(file_path, 'r') as f:
            json_data = json.load(f)
        
        # Initialize a dictionary to store the accumulated sums and counts for each key
        summed_values = {}
        count_values = {}
        
        # Iterate through the dictionaries in the JSON data
        for data_dict in json_data:
            # Accumulate the sums and counts for each key
            for key, value in data_dict.items():
                if key not in summed_values:
                    summed_values[key] = value
                    count_values[key] = 1
                else:
                    summed_values[key] += value
                    count_values[key] += 1
        
        # Calculate the averages for each key
        averaged_values = {key: summed_values[key] / count_values[key] for key in summed_values}
        averaged_values['Percept_ID'] = percept_id
        
        # Append the averaged values to the dataframe
        all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)




C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15600\3346489467.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15600\3346489467.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15600\3346489467.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_15600\3346489467.py:37: FutureWarning: The frame.append method is deprecated and will 

In [ ]:
# Print the resulting dataframe
all_features_cond1 = all_features_cond1.reindex(columns=['Percept_ID'] + list(all_features_cond1.columns[:-1]))
print(all_features_cond1)

In [88]:
# Save the dataframe as JSON
json_file_path = os.path.join(json_dir, 'all_features_cond4.json')
all_features_cond1.to_json(json_file_path, orient='records')

### Plot all averaged values in three conditions

In [ ]:
json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections\\'
)

feat_cond_1 = pd.read_json(str(json_dir) + 'all_features_cond1.json')
feat_cond_2 = pd.read_json(str(json_dir) + 'all_features_cond2.json')
feat_cond_4 = pd.read_json(str(json_dir) + 'all_features_cond4.json')

feat_cond_1

In [93]:
# Get the column values as NumPy arrays
column1_values = feat_cond_1['Percept_ID'].values
column2_values = feat_cond_2['Percept_ID'].values
column3_values = feat_cond_4['Percept_ID'].values

# Find the similar values between the three columns
similar_values = np.intersect1d(column1_values, np.intersect1d(column2_values, column3_values))
print(similar_values)

['Sub005' 'Sub006' 'Sub021' 'Sub025' 'Sub033' 'Sub050']


In [95]:
%matplotlib qt

# Get the column names excluding 'Percept_ID'
columns = [col for col in feat_cond_1.columns if col != 'Percept_ID']

#fig, axes = plt.subplots(1, len(columns), figsize=(25, 5))  # Adjust figsize as needed

fig, axes = plt.subplots(3, 5, figsize=(12, 8))
axes = axes.flatten()
# Iterate over each column and create a boxplot for each dataframe
for i, column in enumerate(columns):
    # Get the data for the column from each dataframe
    data1 = feat_cond_1[column]
    data2 = feat_cond_2[column]
    data3 = feat_cond_3[column]

     # Create a boxplot for the column in the corresponding subplot
    axes[i].boxplot([data1, data2, data3], showfliers=False)
    axes[i].scatter([1]*len(data1), data1, color='red')
    axes[i].scatter([2]*len(data2), data2, color='blue')
    axes[i].scatter([3]*len(data3), data3, color='green')
    axes[i].set_title(column)


    # Create a boxplot for the column in the corresponding subplot
    axes[i].boxplot([data1, data2, data3])
    axes[i].set_title(column)

# Add a legend at the bottom of the plot
legend_labels = ['M1S0', 'M1S1:PreSubharmonic', 'M1S1:Subh-Highest Amp']
legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color) for color in ['red', 'blue', 'green']]
fig.legend(legend_markers, legend_labels, loc='lower center', ncol=3)

# Adjust spacing between subplots and legend
fig.tight_layout(rect=[0, 0.1, 1, 1])

In [96]:
plt.savefig(os.path.join(
    json_dir, 'All_avg_features_conds'
), dpi = 250
)

### Compare motor performance in patients with vs without subharmonic